**CP2410 Assignment 2**  
  
***Introduction***  
This assignment aims to investigate different search tree structures and their applications in Python. The following tests were conducted using the data from the "Santa 2019: Revenge of the Accountants" Kaggle challenge ( https://www.kaggle.com/c/santa-2019-revenge-of-the-accountants ). The goal of this challenge was to assign families to a day for a tour of Santa's workshop without overbooking/underbooking specific days and with minimal accounting fees. To do so, the challenge supplies a list of families and their preferred days, the penalty costs for not giving families their first preference, and a function to cover any other incurred overcrowding costs.  

***Data Structure 1 – Binary Search Trees (from chapter 11.1 of textbook)***  
Binary search trees consist of various nodes connected through parent/children relationships. Each node has a left and right child that hold a key/value pair or a blank. The trait that defines a search tree is that any left child (and all its descendants) should be less than its parent (and all of its ancestors), and any right child (and all its descendants) should be more than its parent (and all of its ancestors). When inserting into this data type, the node will go through a series of less than/greater checks (starting at the root node), then descend the tree through the left or right child accordingly until it reaches a blank node, at which point it will override the blank. This makes searching through this datatype potentially faster than an array, as shown by its big-Oh notation of O(h), where 'h' is the height of the tree. However, if nodes are inserted in descedning/ascending order, the tree will construct with only one type of child, making the height of the tree equal to the number of elements, resulting in O(n) search times.

In the case of this project, binary search trees will be employed in a for loop to conveniently store score values for each family to see which of their preferences results in the lowest overall costs.
  
***Data Structure 2 – AVL Trees (from chapter 11.3 of textbook)***   
AVL trees are binary search trees that restructure themselves to ensure the depth of external nodes do not differ by 2 or more. It does so using trinode restructuring, which takes a trio of problem nodes in a line and rearranges them to a triangle shape. It continues to do so until the conditons of the binary search tree and AVL tree are met. Each trinode restructure takes O(1) time, but the restructuring of an entire tree for the case of an insert/remove can take up to O(lg n). AVL trees have an advantage over binary search trees in that the height of a tree will not expand inconveniently. For example, if the elements were added in ascending order to a binary search tree, the tree would have a height of n, which means that accessing elements would take O(n) time. However, if the same happened in an AVL tree, the tree would be re-arranged as it was developing, leading to a max height of lg n and an O(lg n) time for accessing elements.

In the case of this project, AVL trees will be employed in the same way that binary search trees are, but will hopefully reduce the time taken to access elements.

***Algorithm 1 – Finding minimum via Inorder Traversal***  
Inorder traversal is a way of exploring binary trees, alongside preorder and postorder traversal. Inorder traversal will recursively visit the left child of a node, then the node itself, then the right child. In the case of a search tree, inorder traversal will visit nodes in ascending order, which is an easy way to identify the minimum. Inorder traversal has to evaluate every node in the binary serch tree, and hence, it takes O(n) time

In the case of this project, inorder traversal will be used to return a sorted list of the penalties as to find the minimum.

***Algorithm 2 - Finding minimum directly***  
Using the properties of search trees and inorder traversal, we can specialise/shorten the inorder travesal algorithm to find only the minimum value of the tree instead of a sorted list. In any search tree, the left most element will always be the smallest value. Hence, we can employ the inorder traversal algorithm until the first node is evaluated, at which point we can stop the traversal. In the worst case, the minimum element is stored at the maximum depth, meaning it will have to traverse across the same number of elements as the height of the tree. This will take O(n) for a binary search tree, or O(lg n) for a AVL tree.

In the case of this project, this method will be used to shorten the amount of time taken to find the minimum of the search tree.

## Code Initialisation
Taken from https://www.kaggle.com/inversion/santa-s-2019-starter-notebook

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time

fpath = '../input/santa-2019-revenge-of-the-accountants/family_data.csv'
data = pd.read_csv(fpath, index_col='family_id')
fpath = '../input/santa-2019-revenge-of-the-accountants/sample_submission.csv'
submission = pd.read_csv(fpath, index_col='family_id')
family_size_dict = data[['n_people']].to_dict()['n_people'] 
cols = [f'choice_{i}' for i in range(10)]
choice_dict = data[cols].to_dict()
days = list(range(100, 0, -1))

## Penalty Calculator
Below is my algorithm for calculating the penalty and accounting costs induced from the family/day arrangement.  
This was initially sourced from https://www.kaggle.com/inversion/santa-s-2019-starter-notebook and has mildly modified since obtaining.  
For all code beyond this point, the predicted runtime of each line has been listed as a comment.

In [ ]:
def calc_penalty(table):
    penalty = 0    # 1
    people_scheduled = {k: 0 for k in days}    # n
    for family_id, day in enumerate(table):    # n
        number_of_people = family_size_dict[family_id]    # 2
        people_scheduled[day] += number_of_people    # 3
        # At most, this if statement has to do 10 evaluations, which all take 3
        if day == choice_dict['choice_0'][family_id]:
            penalty += 0    # 2
        elif day == choice_dict['choice_1'][family_id]:
            penalty += 50    # 2
        elif day == choice_dict['choice_2'][family_id]:
            penalty += 50 + 9 * number_of_people    # 4
        elif day == choice_dict['choice_3'][family_id]:
            penalty += 100 + 9 * number_of_people    # 4
        elif day == choice_dict['choice_4'][family_id]:
            penalty += 200 + 9 * number_of_people    # 4
        elif day == choice_dict['choice_5'][family_id]:
            penalty += 200 + 18 * number_of_people    # 4
        elif day == choice_dict['choice_6'][family_id]:
            penalty += 300 + 18 * number_of_people    # 4
        elif day == choice_dict['choice_7'][family_id]:
            penalty += 300 + 36 * number_of_people    # 4
        elif day == choice_dict['choice_8'][family_id]:
            penalty += 400 + 36 * number_of_people    # 4
        elif day == choice_dict['choice_9'][family_id]:
            penalty += 500 + 36 * number_of_people + 199 * number_of_people    # 6
        else:
            penalty += 500 + 36 * number_of_people + 398 * number_of_people    # 6

    for _, occupancy in people_scheduled.items():    # n
        if (occupancy < 125) or (occupancy > 300):    # 2 at worst
            # Use occupancy in penalty to incentivise picking under-occupied days
            penalty += (9999999999 + occupancy*10000)    # 4

    # Calculate the accounting cost
    accounting_cost = (people_scheduled[days[0]] - 125.0) / 400.0 * people_scheduled[days[0]] ** (0.5)    # 9
    accounting_cost = max(0, accounting_cost)    # 3
    yesterday_count = people_scheduled[days[0]]    # 3
    for day in days[1:]: # n
        today_count = people_scheduled[day]    # 2
        diff = abs(today_count - yesterday_count)    # 3
        accounting_cost += max(0, (people_scheduled[day] - 125.0) / 400.0 * people_scheduled[day] ** (0.5 + diff / 50.0))    # 12
        yesterday_count = today_count    # 1

    penalty += accounting_cost    # 2

    return penalty # 1


Evaluating the above, we can see the maximum time it will take is:  
Operations = 1 + n + n(2 + 3 + (10 * 3) + 6) + n(2 + 4) + 9 + 3 + 3 + n(2 + 3 + 12 + 1) + 2 + 1  
Operations = 66n + 19  
Which is O(n)  

## Test 1: Binary Tree

### Class Code

In [ ]:
# Generic tree node class
class TreeNode(object):
    def __init__(self, val, choice_id):
        self.val = val
        self.choice_id = choice_id
        self.left = None
        self.right = None
        self.height = 1

# Binary Tree Class
# The below code was adapted from Bhavya Jain's work
# @ https://www.geeksforgeeks.org/binary-search-tree-set-1-search-and-insertion/
class BinaryTree(object):
    # A utility function to insert a new node with the given key
    def insert(self, root, key, choice_id):
        if not root:
            return TreeNode(key, choice_id)
        elif key < root.val:
            root.left = self.insert(root.left, key, choice_id)
        else:
            root.right = self.insert(root.right, key, choice_id)

        return root

    def inorder(self, root):
        for p in self._subtree_inorder(root):
            yield p

    def _subtree_inorder(self, root):
        if root.left is not None:  # if left child exists, traverse its subtree
            for other in self._subtree_inorder(root.left):
                yield other
        yield root  # visit p between its subtrees
        if root.right is not None:  # if right child exists, traverse its subtree
            for other in self._subtree_inorder(root.right):
                yield other

### Attempt

In [ ]:
submission = pd.read_csv(fpath, index_col='family_id')

start = time.process_time()    # 1

table = submission['assigned_day'].tolist()    # n
new2 = table.copy()    # n
for fam_id, _ in enumerate(new2):    # n
    trial = new2.copy()    # n
    new_scores = list(range(0, len(choice_dict)))    # 5 (3 for range, 1 for len, 1 for assignment)
    myTree = BinaryTree()    # 1
    root = None    # 1
    root = myTree.insert(None, calc_penalty(trial), 10)    # 67n + 20

    for i in new_scores:    # n
        trial[fam_id] = choice_dict[f'choice_{i}'][fam_id]    # 4
        root = myTree.insert(root, calc_penalty(trial), i)    # 67n + 20

    min_id = list(myTree.inorder(root))[0].choice_id    # 2n + 3
    if min_id < 10:    # 1
        new2[fam_id] = choice_dict[f'choice_{min_id}'][fam_id]    # 4

submission['assigned_day'] = new2    # n
score = calc_penalty(new2)    # 66n + 20
print(f'Binary Tree Score: {score}')
print(f'Binary Tree Time: {time.process_time() - start}')
submission.to_csv(f'submission_{score}.csv')

For the worst case for-loop attempt:  
Operations = 1 + n + n + n(n + 5 + 1 + 1 + 1 + 67n + 20 + n(4 + 67n + 20) + 2n + 3 + 1 + 4) + n + 66n + 20  
Operations = 67n<sup>3</sup> + 94n<sup>2</sup> + 105n + 21  
Which is O(n<sup>3</sup>)

## Test 2: AVL Tree

### Class Code

In [ ]:
# AVL tree class
# The below code was adapted from Ajitesh Pathak's work
# @ https://www.geeksforgeeks.org/avl-tree-set-1-insertion/
class AVLTree(object):
    def insert(self, root, key, choice_id):

        # Step 1 - Perform normal BST
        if not root:
            return TreeNode(key, choice_id)
        elif key < root.val:
            root.left = self.insert(root.left, key, choice_id)
        else:
            root.right = self.insert(root.right, key, choice_id)

        # Step 2 - Update the height of the ancestor node
        root.height = 1 + max(self.getHeight(root.left),
                              self.getHeight(root.right))

        # Step 3 - Get the balance factor
        balance = self.getBalance(root)

        # Step 4 - If the node is unbalanced, then try out the 4 cases
            # Case 1 - Left Left
        if balance > 1 and key < root.left.val:
            return self.rightRotate(root)

            # Case 2 - Right Right
        if balance < -1 and key > root.right.val:
            return self.leftRotate(root)

            # Case 3 - Left Right
        if balance > 1 and key > root.left.val:
            root.left = self.leftRotate(root.left)
            return self.rightRotate(root)

            # Case 4 - Right Left
        if balance < -1 and key < root.right.val:
            root.right = self.rightRotate(root.right)
            return self.leftRotate(root)

        return root

    def getHeight(self, root):
        if not root:
            return 0

        return root.height

    def getBalance(self, root):
        if not root:
            return 0

        return self.getHeight(root.left) - self.getHeight(root.right)

    def leftRotate(self, z):

        y = z.right
        T2 = y.left

        # Perform rotation
        y.left = z
        z.right = T2

        # Update heights
        z.height = 1 + max(self.getHeight(z.left),
                           self.getHeight(z.right))
        y.height = 1 + max(self.getHeight(y.left),
                           self.getHeight(y.right))

        # Return the new root
        return y

    def rightRotate(self, z):

        y = z.left
        T3 = y.right

        # Perform rotation
        y.right = z
        z.left = T3

        # Update heights
        z.height = 1 + max(self.getHeight(z.left),
                           self.getHeight(z.right))
        y.height = 1 + max(self.getHeight(y.left),
                           self.getHeight(y.right))

        # Return the new root
        return y

    def preOrder(self, root):

        if not root:
            return

        print("{0} ".format(root.val), end="")
        self.preOrder(root.left)
        self.preOrder(root.right)


    def inorder(self, root):
        for p in self._subtree_inorder(root):
            yield p

    def _subtree_inorder(self, root):
        if root.left is not None:  # if left child exists, traverse its subtree
            for other in self._subtree_inorder(root.left):
                yield other
        yield root  # visit p between its subtrees
        if root.right is not None:  # if right child exists, traverse its subtree
            for other in self._subtree_inorder(root.right):
                yield other
                
    # FOR TEST 3           
    def find_min(self, root):
        if root.left is None:
            return root
        return self.find_min(root.left)

### Attempt

In [ ]:
submission = pd.read_csv(fpath, index_col='family_id')

start = time.process_time()    # 1

table = submission['assigned_day'].tolist()    # n
new2 = table.copy()    # n
for fam_id, _ in enumerate(new2):    # n
    trial = new2.copy()    # n
    new_scores = list(range(0, len(choice_dict)))    # 5 (3 for range, 1 for len, 1 for assignment)
    myTree = AVLTree()    # 1
    root = None    # 1
    root = myTree.insert(None, calc_penalty(trial),  10)    # 66n + log n + 20

    for i in new_scores:    # n
        trial[fam_id] = choice_dict[f'choice_{i}'][fam_id]    # 4
        root = myTree.insert(root, calc_penalty(trial), i)    # 66n + log n + 20

    min_id = list(myTree.inorder(root))[0].choice_id    # 2n + 3
    if min_id < 10:    # 1
        new2[fam_id] = choice_dict[f'choice_{min_id}'][fam_id]    # 4

submission['assigned_day'] = new2    # n
score = calc_penalty(new2)    # 66n + 20
print(f'AVL Tree Score: {score}')
print(f'AVL Tree Time: {time.process_time() - start}')
submission.to_csv(f'submission_{score}.csv')

For the worst case for-loop attempt:  
Operations = 1 + n + n + n(n + 5 + 1 + 1 + 1 + 66n + log n + 20 + n(4 + 66n + log n + 20) + 2n + 3 + 1 + 4) + n + 66n + 20  
Operations = 66n<sup>3</sup> + n<sup>2</sup>log n + 93n<sup>2</sup> + n log n + 105n + 21  
Which is O(n<sup>3</sup>)

## Test 3: find_min() function

### Attempt

In [ ]:
submission = pd.read_csv(fpath, index_col='family_id')

start = time.process_time()    # 1

table = submission['assigned_day'].tolist()    # n
new2 = table.copy()    # n
for fam_id, _ in enumerate(new2):    # n
    trial = new2.copy()    # n
    new_scores = list(range(0, len(choice_dict)))    # 5 (3 for range, 1 for len, 1 for assignment)
    myTree = AVLTree()    # 1
    root = None    # 1
    root = myTree.insert(None, calc_penalty(trial),  10)    # 66n + log n + 20

    for i in new_scores:    # n
        trial[fam_id] = choice_dict[f'choice_{i}'][fam_id]    # 4
        root = myTree.insert(root, calc_penalty(trial), i)    # 66n + log n + 20

    min_id = myTree.find_min(root).choice_id    # log n + 2
    if min_id < 10:    # 1
        new2[fam_id] = choice_dict[f'choice_{min_id}'][fam_id]    # 4

submission['assigned_day'] = new2    # n
score = calc_penalty(new2)    # 48n + 3
print(f'AVL Tree Score: {score}')
print(f'AVL Tree Time: {time.process_time() - start}')
submission.to_csv(f'submission_{score}.csv')

For the worst case for-loop attempt:  
Operations = 1 + n + n + n(n + 5 + 1 + 1 + 1 + 66n + log n + 20 + n(4 + 66n + log n + 20) + log n + 2 + 1 + 4) + n + 66n + 20  
Operations = 66n<sup>3</sup> + n<sup>2</sup>log n + 91n<sup>2</sup> + 2n log n + 104n + 21  
Which is O(n<sup>3</sup>)

## Comparisons
The analysis has shown that all of the above algorithms have a O(n<sup>3</sup>) time, which means they will scale similarly with the number of inputs. Looking at the results, test 1 with binary search trees took 437s, test 2 with AVL trees took 445s and test 3 with AVL trees and an optimised minimum function took 443s. The O(n) equations for each test have been plotted below for comparison.

In [ ]:
# Import our modules that we are using
import matplotlib.pyplot as plt
from math import log2

# Create the vectors X and Y
x = np.array(range(100),dtype='int64')
y_1 = np.array(range(100),dtype='int64')
y_2 = np.array(range(100),dtype='int64')
y_3 = np.array(range(100),dtype='int64')

for j in x:
    if j != 0:
        y_1[j] = 67*(j**3) + 94*(j**2) + 105*j + 21
        y_2[j] = 66*(j**3) + (j**2)*log2(j) + 93*(j**2) + j*log2(j) + 105*j + 21
        y_3[j] = 66*(j**3) + (j**2)*log2(j) + 91*(j**2) + 2*j*log2(j) + 104*j + 21


# Create the plot
plt.plot(x,y_1,label='Binary Search Tree')
plt.plot(x,y_2,label='AVL Tree')
plt.plot(x,y_3,label='AVL Tree (find_min)')


# Add a title
plt.title('Runtime comparison')

# Add X and y Label
plt.xlabel('Inputs')
plt.ylabel('# of primitive operations')

# Add a Legend
plt.legend()

In [ ]:
# Estimated units of time taken for each algorithm for an n of 6000
y_1 = 67*(6000**3) + 94*(6000**2) + 105*6000 + 21
print("Binary serach tree: "+str(y_1))
y_2 = 66*(6000**3) + (6000**2)*log2(6000) + 93*(6000**2) + 6000*log2(6000) + 105*6000 + 21
print("AVL tree: "+str(y_2))
y_3 = 66*(6000**3) + (6000**2)*log2(6000) + 91*(6000**2) + 2*6000*log2(6000) + 104*6000 + 21
print("AVL tree (find_min): "+str(y_3))

As shown above, the different alogrithms did not have a significant effect on the final execution time.  

The binary search tree performed slightly better than the AVL tree in practice (as opposed to the big-Oh analysis) for a few reasons. The first of which was that alot of families (especially at the beginning of execution) had their preference scores handed to the array in ascending order. In this scenario, the binary search tree would have continued to append right children in O(h) time, however, the AVL tree would have tried to rearrange the tree using trinode restructing multiple times throughout the construction. Both the insertion and the restructure process take O(lg n) each, which resulted in it breaking even with the binary search tree in terms of construction time. If nodes of this tree were continuously accessed, the AVL tree would have saved more time. However, in this scenario, only inorder traversal was used on the trees, which takes O(n) time regardless of its structure.  

In an attempt to utilise the height rebalancing of the AVL tree, the inorder traversal algorithm was reworked to process only up until the leftmost element was reached. While both in theory and practice, the runtime was shortened, it had a very mild effect. This is because this part of the process takes a very small portion of the overall time needed to complete the program (when compared to the calc_penalty function).

To conclude, binary search trees are an effecitve sorting structure that can be easily applied to this scenario. In the case that a search tree was continually accessed, an AVL tree would prove to be a better choice, as alot of the binary search trees are unbalanced and lead to longer big-Oh times. The adjusted find_min algorithm is also effective on reducing runtime, but in the case of this study, did not have a meaningful impact on the overall runtime as other sections of the code were very time-consuming.